# Praktikum 1
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [2]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

## Preprocessing

In [4]:
df = pd.read_csv('train.csv')
df

,Unnamed: 0,Unnamed: 0.1,time,frontal,vertical,lateral,id,rssi,phase,freq,label,male
0,0,0,0.00,0.272030,1.00820,-0.082102,1,-63.5,2.42520,924.25,1,0
1,1,1,0.50,0.272030,1.00820,-0.082102,1,-63.0,4.73690,921.75,1,0
2,2,2,1.50,0.447910,0.91636,-0.013684,1,-63.5,3.03110,923.75,1,0
3,3,3,1.75,0.447910,0.91636,-0.013684,1,-63.0,2.03710,921.25,1,0
4,4,4,2.50,0.342380,0.96229,-0.059296,1,-63.5,5.89200,920.25,1,0
5,5,5,3.25,0.342380,0.96229,-0.059296,4,-56.5,1.25630,924.75,1,0
6,6,6,4.00,0.307210,0.99674,-0.070699,1,-63.5,3.87180,922.75,1,0
7,7,7,5.00,0.272030,0.99674,-0.093505,1,-64.0,2.49430,924.25,1,0
8,8,8,5.50,0.236850,1.03120,-0.127710,1,-64.5,2.00800,924.75,1,0
9,9,9,6.00,0.236850,1.03120,-0.127710,1,-66.0,1.19040,925.75,1,0


In [15]:
targets = df['label']
features = df.drop(columns=['label', 'Unnamed: 0', 'Unnamed: 0.1'])
features.head()

,time,frontal,vertical,lateral,id,rssi,phase,freq,male
0,0.00,0.27203,1.00820,-0.082102,1,-63.5,2.4252,924.25,0
1,0.50,0.27203,1.00820,-0.082102,1,-63.0,4.7369,921.75,0
2,1.50,0.44791,0.91636,-0.013684,1,-63.5,3.0311,923.75,0
3,1.75,0.44791,0.91636,-0.013684,1,-63.0,2.0371,921.25,0
4,2.50,0.34238,0.96229,-0.059296,1,-63.5,5.8920,920.25,0


In [16]:
def normalize(data) :
    min_data = data.min()
    max_data = data.max()
    result = list(map(lambda x: (x-min_data)/(max_data-min_data)*1, data))
    return result

print (features.columns)
features['freq'] = normalize(features['freq'])
features['phase'] = normalize(features['phase'])
features['rssi'] = normalize(features['rssi'])
features['time'] = normalize(features['time'])
# features['id'] = pd.Categorical(features['id'])
dfId = pd.get_dummies(features['id'], prefix = 'id')

features = pd.concat([features, dfId], axis=1)
features = features.drop(columns=['id'], axis=1)

features

Index(['time', 'frontal', 'vertical', 'lateral', 'id', 'rssi', 'phase', 'freq',
       'male'],
      dtype='object')


,time,frontal,vertical,lateral,rssi,phase,freq,male,id_1,id_2,id_3,id_4
0,0.000000,0.272030,1.00820,-0.082102,0.253731,0.386074,0.727273,0,1,0,0,0
1,0.000317,0.272030,1.00820,-0.082102,0.268657,0.754079,0.272727,0,1,0,0,0
2,0.000950,0.447910,0.91636,-0.013684,0.253731,0.482529,0.636364,0,1,0,0,0
3,0.001109,0.447910,0.91636,-0.013684,0.268657,0.324291,0.181818,0,1,0,0,0
4,0.001584,0.342380,0.96229,-0.059296,0.253731,0.937963,0.000000,0,1,0,0,0
5,0.002059,0.342380,0.96229,-0.059296,0.462687,0.199994,0.818182,0,0,0,0,1
6,0.002534,0.307210,0.99674,-0.070699,0.253731,0.616362,0.454545,0,1,0,0,0
7,0.003168,0.272030,0.99674,-0.093505,0.238806,0.397074,0.727273,0,1,0,0,0
8,0.003485,0.236850,1.03120,-0.127710,0.223881,0.319659,0.818182,0,1,0,0,0
9,0.003802,0.236850,1.03120,-0.127710,0.179104,0.189503,1.000000,0,1,0,0,0


## Model

### RNN

In [ ]:
rnn = Sequential()

### LSTM

In [ ]:
lstm = Sequential()

### CNN

In [ ]:
cnn = Sequential()

## Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
f1_score_test = f1_score(y_true, y_pred, average='macro')
accuracy_score_test = accuracy_score(y_true, y_pred)
confusion_matrix_test = confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
np.set_printoptions(precision=2)
print('Confusion matrix')
print(cm)
plt.figure()
plot_confusion_matrix(confusion_matrix_test)